# 02 - Nettoyage et Préparation des Données

**Objectif** : Produire un jeu de données propre, cohérent et exploitable pour l'analyse et le reporting.

**Étapes** :
- Traiter les valeurs manquantes (suppression, imputation et justification).
- Corriger ou exclure les valeurs incohérentes.
- Harmoniser les formats (dates, géographie, catégories).
- Documenter les règles de transformation.

**Livrables** : Dataset final nettoyé, scripts reproductibles, documentation synthétique.

## 1. Import des Librairies et Chargement des Données

In [ ]:
# Import des librairies nécessaires
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Chemins des fichiers de données
data_files = {
    'demandes': '../data/demandes_service_public.csv',
    'documents': '../data/documents_administratifs_ext.csv',
    'centres': '../data/centres_service.csv',
    'communes': '../data/details_communes.csv',
    'socioeco': '../data/donnees_socioeconomiques.csv',
    'logs': '../data/logs_activite.csv',
    'reseau': '../data/reseau_routier_togo_ext.csv',
    'developpement': '../data/developpement.csv'
}

# Chargement des datasets
datasets = {}
for name, path in data_files.items():
    try:
        datasets[name] = pd.read_csv(path, sep=',', encoding='utf-8')
        print(f"✅ {name} chargé : {datasets[name].shape[0]} lignes, {datasets[name].shape[1]} colonnes")
    except Exception as e:
        print(f"❌ Erreur chargement {name} : {e}")

# Aperçu rapide
for name, df in datasets.items():
    print(f"\n=== {name.upper()} ===")
    print(df.head(2))

## 2. Traitement des Valeurs Manquantes

**Stratégie** :
- Identifier les NaN par dataset.
- Suppression si <5% et non critiques.
- Imputation par moyenne/médiane/mode selon le type de variable.
- Justification : Basé sur l'analyse EDA (ex. : médiane pour délais, mode pour catégories).

In [ ]:
# Fonction pour analyser et traiter les NaN
def handle_missing_values(df, name):
    print(f"\n=== Analyse NaN pour {name} ===")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    print(missing_pct[missing_pct > 0])

    # Exemples de traitement (adapter selon EDA)
    if name == 'demandes':
        # Imputation médiane pour délais (distribution asymétrique)
        if 'delai_effectif' in df.columns:
            df['delai_effectif'].fillna(df['delai_effectif'].median(), inplace=True)
            print("✅ Imputation médiane pour 'delai_effectif'")
        # Suppression si NaN critiques (ex. : ID)
        df.dropna(subset=['id_demande'], inplace=True)
        print("✅ Suppression des lignes sans 'id_demande'")

    elif name == 'centres':
        # Mode pour catégories
        df.fillna(df.mode().iloc[0], inplace=True)
        print("✅ Imputation mode pour catégories")

    # Autres datasets : suppression si <1%
    if missing_pct.sum() < 1:
        df.dropna(inplace=True)
        print("✅ Suppression des NaN (<1%)")

    return df

# Appliquer à tous les datasets
for name in datasets:
    datasets[name] = handle_missing_values(datasets[name], name)

print("\n✅ Traitement NaN terminé pour tous les datasets.")

## 3. Correction des Valeurs Incohérentes

**Stratégie** :
- Détecter outliers via IQR (comme en EDA).
- Corriger ou exclure (ex. : winsorisation pour capacités extrêmes).
- Vérifier cohérence logique (ex. : âge >0, dates plausibles).

In [ ]:
# Fonction pour détecter et corriger outliers (IQR)
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

# Fonction pour corriger incohérences
def correct_inconsistencies(df, name):
    print(f"\n=== Correction incohérences pour {name} ===")

    if name == 'demandes':
        # Délais négatifs ou >100 jours = incohérents
        if 'delai_effectif' in df.columns:
            df = df[(df['delai_effectif'] > 0) & (df['delai_effectif'] <= 100)]
            print("✅ Suppression délais incohérents (>100j ou <0)")

        # Âge entre 0 et 120
        if 'age_demandeur' in df.columns:
            df = df[(df['age_demandeur'] >= 0) & (df['age_demandeur'] <= 120)]
            print("✅ Filtrage âge plausible")

    elif name == 'centres':
        # Capacité : winsorisation des outliers
        if 'capacite_jour' in df.columns:
            lb, ub = detect_outliers_iqr(df, 'capacite_jour')
            df['capacite_jour'] = np.clip(df['capacite_jour'], lb, ub)
            print("✅ Winsorisation capacité (IQR)")

    elif name == 'communes':
        # Densité population >0
        if 'densite_pop' in df.columns:
            df = df[df['densite_pop'] > 0]
            print("✅ Suppression densités négatives")

    print(f"Dimensions après correction : {df.shape}")
    return df

# Appliquer à tous
for name in datasets:
    datasets[name] = correct_inconsistencies(datasets[name], name)

print("\n✅ Corrections incohérences terminées.")

## 4. Harmonisation des Formats

**Stratégie** :
- Dates : Conversion en datetime, format uniforme.
- Géographie : Normalisation noms (minuscules, accents).
- Catégories : Mapping uniforme (ex. : 'LOME' → 'Lomé').

In [ ]:
# Fonction pour harmoniser formats
def harmonize_formats(df, name):
    print(f"\n=== Harmonisation formats pour {name} ===")

    # Dates
    date_cols = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
    for col in date_cols:
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            print(f"✅ Conversion datetime pour {col}")
        except:
            print(f"⚠️ Échec conversion {col}")

    # Géographie : normalisation noms
    geo_cols = [col for col in df.columns if 'commune' in col.lower() or 'region' in col.lower() or 'ville' in col.lower()]
    for col in geo_cols:
        df[col] = df[col].astype(str).str.lower().str.strip()
        # Mapping spécifique (exemple)
        mapping = {'lome': 'lomé', 'kara': 'kara', 'sokode': 'sokodé'}
        df[col] = df[col].replace(mapping)
        print(f"✅ Normalisation géo pour {col}")

    # Catégories : uniformisation
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        df[col] = df[col].str.lower().str.strip()
        print(f"✅ Uniformisation catégories pour {col}")

    return df

# Appliquer
for name in datasets:
    datasets[name] = harmonize_formats(datasets[name], name)

print("\n✅ Harmonisation formats terminée.")

## 5. Documentation des Transformations Appliquées

**Règles appliquées** :
- **Valeurs manquantes** : Imputation médiane pour variables numériques asymétriques (délais), mode pour catégories, suppression si <1% ou critiques (ID).
- **Incohérences** : Suppression valeurs hors plages logiques (délais >100j, âge >120), winsorisation IQR pour outliers capacitaires.
- **Formats** : Conversion datetime pour colonnes temporelles, normalisation minuscules/accents pour géo, uniformisation catégories.

**Justifications** :
- Basé sur EDA : Médiane préserve distribution, suppression minimise perte de données.
- Cohérence métier : Plages plausibles pour éviter biais.

**Impact** : Réduction NaN de X% à Y%, correction Z outliers, harmonisation W colonnes.

## 6. Sauvegarde du Dataset Final

**Dataset final** : Jointure unifiée des datasets nettoyés pour analyse/KPI.

In [ ]:
# Jointure finale (exemple basé sur EDA)
# Adapter selon clés réelles (ex. : region, commune)
merged = datasets['demandes'].merge(datasets['centres'], on='region', how='left') \
                             .merge(datasets['communes'], on='commune', how='left') \
                             .merge(datasets['socioeco'], on='region', how='left')

print(f"Dataset final : {merged.shape[0]} lignes, {merged.shape[1]} colonnes")

# Sauvegarde
merged.to_csv('../data/dataset_nettoye.csv', index=False, encoding='utf-8')
print("✅ Dataset sauvegardé : ../data/dataset_nettoye.csv")

# Aperçu
print(merged.head())